# 청약 applyhome

In [1]:
pwd

'C:\\Users\\황혜수\\Desktop\\github\\24f-houseplan\\3_applyhome\\2_embedding'

In [2]:
!pip install pdfplumber


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#[API] metadata 구축완료

In [4]:
import os
import pdfplumber
import pandas as pd
import re
import random

# 현재 작업 디렉토리 확인
current_dir = os.getcwd()

# 상대 경로 설정
api_data_path = os.path.join(current_dir, "api_data")

# PDF에서 표를 추출하여 데이터프레임으로 변환하는 함수 (특정 키워드 필터링)
def extract_filtered_tables_from_pdf(pdf_path, keyword="공급금액"):
    try:
        tables = []
        with pdfplumber.open(pdf_path) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):
                page_tables = page.extract_tables()
                for table in page_tables:
                    df = pd.DataFrame(table[1:], columns=table[0])  # 첫 번째 행을 컬럼으로 설정
                    if any(keyword in str(cell) for row in table for cell in row):
                        tables.append(df)
        return tables
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return []

# PDF 파일 읽기 함수
def read_pdf_files(directory_path):
    pdf_files = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

# 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    try:
        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return None

# 메타데이터 추출 함수
def extract_metadata(text, max_supply_price):
    metadata = {
        "supply_name": None,
        "region_name": None,
        "application_schedule": None,
        "special_supply_conditions": [],
        "enter_day": None,
        "max_supply_price": max_supply_price
    }

    # 공급명 추출
    supply_name_match = re.search(r"입주자모집공고주요정보\s*(.+)", text)
    if supply_name_match:
        metadata["supply_name"] = supply_name_match.group(1).strip()

    # 지역명 추출
    region_name_match = re.search(r"공급위치\s*(.+)", text)
    if region_name_match:
        metadata["region_name"] = region_name_match.group(1).strip()

    # 청약일정 추출
    #dates = re.findall(r"\d{4}-\d{2}-\d{2}", text)
    #if dates:
    #    metadata["application_schedule"] = max(dates)

    # 청약일정 추출
    schedule_match = re.search(r"당첨자 발표일\s*(\d{4}-\d{2}-\d{2})", text)
    if schedule_match:
        metadata["application_schedule"] = schedule_match.group(1).strip()

    
    # 특별공급조건 추출
    special_conditions_keywords = ["다자녀", "신혼부", "생애최", "노부모", "신생아", "청년"]
    for keyword in special_conditions_keywords:
        if keyword in text:
            metadata["special_supply_conditions"].append(keyword)

    # 입주예정월 추출
    enter_day_match = re.search(r"입주예정월 :\s*(\d{4}\.\d{2})", text)
    if enter_day_match:
        metadata["enter_day"] = enter_day_match.group(1).strip()

    return metadata

# PDF 파일 가져오기
api_pdfs = read_pdf_files(api_data_path)

# API PDF 메타데이터 추출
api_metadata = []
if api_pdfs:
    for pdf_path in api_pdfs:
        print(f"Reading API PDF: {pdf_path}")
        text = extract_text_from_pdf(pdf_path)

        max_supply_price = None
        filtered_tables = extract_filtered_tables_from_pdf(pdf_path)
        for table in filtered_tables:
            string_values = table.stack().apply(lambda x: x if isinstance(x, str) else None).dropna()
            split_values = string_values.str.split(expand=True).stack()
            numeric_values = split_values.apply(lambda x: int(x.replace(",", "")) if x.replace(",", "").isdigit() else None).dropna()
            if not numeric_values.empty:
                max_supply_price = numeric_values.min()
                break

        if text:
            metadata = extract_metadata(text, max_supply_price)
            api_metadata.append(metadata)

# 메타데이터 출력
for i, metadata in enumerate(api_metadata):
    print(f"\nMetadata {i+1}:")
    for key, value in metadata.items():
        print(f"{key}: {value}")


Reading API PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\api_data\1.pdf
Reading API PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\api_data\2.pdf
Reading API PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\api_data\3.pdf
Reading API PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\api_data\4.pdf
Reading API PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\api_data\5.pdf
Reading API PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\api_data\6.pdf
Reading API PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\api_data\7.pdf
Reading API PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\api_data\8.pdf
Reading API PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\api_data\9.pdf

Metadata 1:
supply_name: 성남금토 A-4블록 신혼희망타운(공공분양)(본청약)
region_name: 경기도 성남시 수정구 금토동 일원 성남금토 공공주택지구 내 A-4블록
application

In [5]:
import csv

# CSV 파일 저장 함수
def save_metadata_to_csv(metadata_list, output_file):
    try:
        # 메타데이터의 키를 가져와 CSV 헤더로 설정
        fieldnames = metadata_list[0].keys() if metadata_list else []
        
        with open(output_file, mode='w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # 헤더 작성
            writer.writeheader()
            
            # 메타데이터 작성
            for metadata in metadata_list:
                writer.writerow(metadata)
        
        print(f"Metadata successfully saved to {output_file}")
    except Exception as e:
        print(f"Error saving metadata to CSV: {e}")

# CSV 저장 경로 설정
output_csv_file = "applyhome_dataprocessing_apidata.csv"

# 메타데이터 저장
save_metadata_to_csv(api_metadata, output_csv_file)


Metadata successfully saved to applyhome_dataprocessing_apidata.csv


In [7]:
import os
import pdfplumber
import random

# 현재 작업 디렉토리 확인
current_dir = os.getcwd()

# 상대 경로 설정
crawl_data_path = os.path.join(current_dir, "crawl_data")

# PDF 파일 읽기 함수
def read_pdf_files(directory_path):
    pdf_files = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

# 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    try:
        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return None

# PDF 파일 가져오기
crawl_pdfs = read_pdf_files(crawl_data_path)

# 무작위 파일 읽기 (Crawl Data)
if crawl_pdfs:
    random_crawl_pdf = random.choice(crawl_pdfs)
    print(f"\nRandomly selected Crawl PDF: {random_crawl_pdf}")
    crawl_text = extract_text_from_pdf(random_crawl_pdf)
    if crawl_text:
        print(f"\nExtracted Text from Crawl PDF:\n{crawl_text}\n")
else:
    print("No PDF files found in the specified directory.")



Randomly selected Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\crawl_data\11.pdf

Extracted Text from Crawl PDF:
25. 1. 6. 오후 8:55 LH청약플러스
공고문
소득계산방법  청약신청 바로가기  목록
(신규)과천지식정보타운 S-11BL 행복주택(리츠) 입주자 모집공고('24. 12. 20.)
공고상태:접수중 유형:행복주택 공고일:2024.12.20
공고문
(241220)과천지식정보타운_S11BL_행복주택(리츠)_입주자_모집_공고문.hwp 바로보기
(241220)과천지식정보타운_S11BL_행복주택(리츠)_입주자_모집_공고문.pdf 바로보기
다운로드
(팸플릿)과천지식정보타운S11BL_행복주택(리츠).pdf 바로보기
붙임1)필수_제출서류(양식1_양식4).hwp 바로보기
붙임2)(대학생_복학예정자만제출)확인서(양식5).hwp 바로보기
붙임3)(예비신혼부부계층만_제출)신청및세대구성_확인서(양식6_7).hwp 바로보기
붙임4)위임장_및_편의시설_설치신청_양식(필요하신분만작성).zip
문서뷰어다운로드 한컴오피스 뷰어  Acrobat Reader  DWG 뷰어 
공급정보
과천지식정보타운 S-11BL 행복주택(리츠)
소재지 : 경기도 과천시 갈현동 593-1
전용면적(㎡) : 16.8~44.37 총 세대수 : 846 난방방식 : 지역난방 입주예정월 : 2026.05
단지 관련 이미지 정보
이미지를 클릭하시면 큰 화면으로 보실 수 있습니다.
평면도  위치도  토지이용계획도  단지조감도  단지배치도 
 16A형(고령자) 16A형(대학생, 청년) 16A형(주거급여수급자) 26AS형(고령자-주거약자용) 26A형(대학생, 청년) 26A형(주거급여수급자) 36A형(고령자) 36A형(신혼
16A형(고령자)
상기 이미지는 소비자의 이해를 돕기 위해 제작된 것으로 실제와 다를 수 있습니다.
단지 관련 위치 정

In [8]:
import os
import pdfplumber
import re

# 현재 작업 디렉토리 확인
current_dir = os.getcwd()

# 상대 경로 설정
crawl_data_path = os.path.join(current_dir, "crawl_data")

# PDF 파일 읽기 함수
def read_pdf_files(directory_path):
    pdf_files = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

# 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    try:
        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return None

# 메타데이터 추출 함수
def extract_metadata(text):
    metadata = {
        "supply_name": None,
        "region_name": None,
        "supply_type": None,
        "area": None,
        "application_schedule": None,
        "special_supply_conditions": []
    }

    # 공급명 추출
    supply_name_match = re.search(r"(.+?)\.pdf 바로보기", text)
    if supply_name_match:
        metadata["supply_name"] = supply_name_match.group(1).strip()

    # 지역명 추출
    region_name_match = re.search(r"모집지역\s*:\s*(.+?)$|소재지\s*:\s*(.+?)$", text, re.MULTILINE)
    if region_name_match:
        region_name = region_name_match.group(1) or region_name_match.group(2)
        if "확인" not in region_name:
            metadata["region_name"] = region_name.strip()

    # 공급유형 추출
    supply_type_match = re.search(r"유형\s*:\s*(.+?)\s", text)
    if supply_type_match:
        metadata["supply_type"] = supply_type_match.group(1).strip()

    # 면적 추출
    area_match = re.search(r"전용면적\(㎡\)\s*:\s*(\d+\.\d+)", text)
    if area_match:
        metadata["area"] = area_match.group(1).strip()

    # 청약일정 추출
    schedule_match = re.search(r"접수기간\s*:\s*(\d{4}\.\d{2}\.\d{2})\s*~\s*(\d{4}\.\d{2}\.\d{2})", text)
    if schedule_match:
        metadata["application_schedule"] = schedule_match.group(2).strip()

    # 특별공급조건 추출
    special_conditions_keywords = ["다자녀", "신혼", "생애", "노부모", "신생아", "청년"]
    for keyword in special_conditions_keywords:
        if keyword in text:
            metadata["special_supply_conditions"].append(keyword)

    return metadata

# PDF 파일 가져오기
crawl_pdfs = read_pdf_files(crawl_data_path)

# Crawl PDF 메타데이터 추출
crawl_metadata = []
if crawl_pdfs:
    for pdf_path in crawl_pdfs:
        print(f"Reading Crawl PDF: {pdf_path}")
        text = extract_text_from_pdf(pdf_path)
        if text:
            metadata = extract_metadata(text)
            crawl_metadata.append(metadata)

# 메타데이터 출력
for i, metadata in enumerate(crawl_metadata):
    print(f"\nMetadata {i+1}:")
    for key, value in metadata.items():
        print(f"{key}: {value}")


Reading Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\crawl_data\1.pdf
Reading Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\crawl_data\10.pdf
Reading Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\crawl_data\11.pdf
Reading Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\crawl_data\12.pdf
Reading Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\crawl_data\13.pdf
Reading Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\crawl_data\14.pdf
Reading Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\crawl_data\15.pdf
Reading Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\crawl_data\16.pdf
Reading Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\crawl_data\17.pdf
Reading Crawl PDF: C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2

In [10]:
# 메타데이터를 DataFrame으로 변환
metadata_df = pd.DataFrame(crawl_metadata)

# CSV 파일로 저장
output_csv_path = os.path.join(current_dir, "applyhome_dataprocessing_crawldata.csv")
metadata_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
print(f"\nMetadata has been saved to {output_csv_path}")


Metadata has been saved to C:\Users\황혜수\Desktop\github\24f-houseplan\3_applyhome\2_embedding\applyhome_dataprocessing_crawldata.csv


In [9]:
import pandas as pd

def filter_policies_by_age_and_period(documents, user_age, current_date):
    """
    정책 문서 리스트를 사용자 연령 및 현재 날짜를 기준으로 필터링
    """
    filtered_documents = []

    for doc in documents:
        metadata = doc.metadata

        # 1. 연령대 필터링
        age_filter = metadata.get("연령", "제한없음")

        # 연령대가 "제한없음"인 경우 바로 추가
        if age_filter == "제한없음":
            filtered_documents.append(doc)
            continue

        # 사용자 나이가 범위에 포함되지 않으면 제외
        if not is_age_within_range(user_age, age_filter):
            continue

        # 2. 사업 신청 기간 필터링
        application_period = metadata.get("사업 신청 기간", "상시")

        # "상시"와 "미정"인 경우 조건 만족으로 간주
        if application_period in ["상시", "미정"]:
            filtered_documents.append(doc)
            continue

        # 날짜 범위 필터링
        if isinstance(application_period, tuple):
            if not is_within_date_range(current_date, application_period):
                continue

        # 조건에 맞는 문서를 추가
        filtered_documents.append(doc)

    return filtered_documents


def is_age_within_range(user_age, age_filter):
    """
    연령 필터링: "18 ~ 34" 형식 처리
    """
    try:
        # 숫자 범위 처리
        if "~" in age_filter:
            start_age, end_age = map(int, age_filter.split("~"))
            user_age = int(user_age)
            return start_age <= user_age <= end_age

        return False  # 잘못된 형식인 경우 False 반환
    except Exception as e:
        print(f"Error processing age range: {e}")
        return False


def is_within_date_range(current_date, application_period):
    """
    사업 신청 기간 필터링: (start_date, end_date) 형식 처리
    """
    try:
        # application_period가 튜플인지 확인
        if isinstance(application_period, tuple) and len(application_period) == 2:
            start_date, end_date = application_period

            # start_date 또는 end_date가 NaT인 경우 False 반환
            if pd.isna(start_date) or pd.isna(end_date):
                return False

            # 현재 날짜가 범위 내에 있는지 확인
            return start_date <= current_date <= end_date

        return False  # 잘못된 형식인 경우 False 반환
    except Exception as e:
        print(f"Error processing application period: {e}")
        return False


In [10]:
# 테스트 실행
current_date = pd.Timestamp("2025-01-03")  # 오늘 날짜
user_age = 25  # 사용자 나이

filtered_docs = filter_policies_by_age_and_period(policy_documents, user_age, current_date)




In [11]:
import pandas as pd

def is_within_date_range(current_date, application_period):
    """
    사업 신청 기간 필터링: 문자열로 표현된 (start_date, end_date) 형식 처리
    """
    try:
        # application_period가 문자열인 경우
        if isinstance(application_period, str):
            # eval()에 안전하게 Timestamp와 NaT를 전달
            application_period = eval(application_period, {"Timestamp": pd.Timestamp, "NaT": pd.NaT})
        
        # 튜플인지 확인
        if isinstance(application_period, tuple) and len(application_period) == 2:
            start_date, end_date = application_period

            # start_date와 end_date가 Timestamp인지 확인
            if not isinstance(start_date, pd.Timestamp):
                start_date = pd.Timestamp(start_date)
            if not isinstance(end_date, pd.Timestamp):
                end_date = pd.Timestamp(end_date)

            # start_date 또는 end_date가 NaT인 경우 False 반환
            if pd.isna(start_date) or pd.isna(end_date):
                return False

            # 날짜 비교
            return start_date <= current_date <= end_date

        return False  # 잘못된 형식인 경우 False 반환
    except Exception as e:
        print(f"Error processing application period: {e}")
        return False


def filter_policies_by_age_and_period(documents, user_age, current_date):
    """
    정책 문서 리스트를 사용자 연령 및 현재 날짜를 기준으로 필터링
    """
    filtered_documents = []

    for doc in documents:
        metadata = doc.metadata

        # 1. 연령대 필터링
        age_filter = metadata.get("연령", "제한없음")
        if age_filter != "제한없음" and not is_age_within_range(user_age, age_filter):
            #print(f"Document excluded due to age filter: {metadata}")
            continue

        # 2. 사업 신청 기간 필터링
        application_period = metadata.get("사업 신청 기간", "상시")
        if application_period not in ["상시", "미정"]:
            if not is_within_date_range(current_date, application_period):
                #print(f"Document excluded due to date range: {metadata}")
                continue

        # 조건에 맞는 문서를 추가
        filtered_documents.append(doc)

    return filtered_documents


def is_age_within_range(user_age, age_filter):
    """
    연령 필터링: "18 ~ 34" 형식 처리
    """
    try:
        if "~" in age_filter:
            start_age, end_age = map(int, age_filter.split("~"))
            user_age = int(user_age)
            return start_age <= user_age <= end_age

        return False
    except Exception as e:
        print(f"Error processing age range: {e}")
        return False


In [12]:
current_date = pd.Timestamp("2025-01-03")
user_age = 60

filtered_docs = filter_policies_by_age_and_period(policy_documents, user_age, current_date)


# 결과 출력
print(len(filtered_docs))
for doc in filtered_docs:
    print(f" 정책 설명: {doc.text},메타데이터: {doc.metadata}")  
    print("----") 

14
 정책 설명: 정책명: 대전 전세사기피해자 지원
설명: 대전광역시 전세사기피해자 주거안정 및 피해회복 지원
※ 당해년도 예산이 소진되어 신청하신 지원금은 2025년도에 지급될 예정입니다.
지원 내용: □ 주거안정지원금(최대 100만원)
ㅇ 가구원수에 따라 차등 지급(1인 가구: 60만원, 2인 가구: 80만원, 3인이상 가구: 100만원)
ㅇ 가구원수는 주민등록에 생계를 같이하는 가족(친족)
□ 이사비용(공공임대주택 입주시, 최대 100만원)
ㅇ 관내 공공임대주택 입주자가 이사업체에 지불한 비용(이사비, 사다리차 이용비, 에어컨 이전설치비 등)
□ 월세(민간주택 이주시, 최대 480만원)
ㅇ 경매로 인해 피해주택에서 대전시 새로운주택으로 이사한 피해자가 임대차계약서상 임대인에게 지불한 월차임 (관리비, 공과금 등은 제외)
ㅇ 연속한 12개월, 월 40만원 이하 / 2회 분할 신청
사업 신청 기간: 상시
거주지 및 소득: ❍ 지원대상 : 전세사기피해를 입은 임차인(특별법의 전세사기피해자등, 특별법 시행전 주택도시보증공사에서 전세피해확인서 발급자) ❍ 지원요건 : 전세사기피해자 결정일 현재 ① 임대차계약을 체결한 피해주택의 소재지가 대전시이고, ② 대전시에 주민등록을 두고 거주하는 자,메타데이터: {'정책명': '대전 전세사기피해자 지원', '사업 신청 기간': '상시', '연령': '제한없음', '거주지': '❍ 지원대상 : 전세사기피해를 입은 임차인(특별법의 전세사기피해자등, 특별법 시행전 주택도시보증공사에서 전세피해확인서 발급자) ❍ 지원요건 : 전세사기피해자 결정일 현재 ① 임대차계약을 체결한 피해주택의 소재지가 대전시이고, ② 대전시에 주민등록을 두고 거주하는 자', '운영 기관': '대전 전세피해지원센터', '주관 기관': '대전 전세피해지원센터', '특화 분야': '제한없음', '취업 상태': '제한없음', '학력': '제한없음'}
----
 정책 설명: 정책명: 광산 청년온(溫)가(家) 운영
설명: 지역 청년의 성공적인 독립을 지원하기 위하여 

In [17]:
filtered_docs

[Document(id_='7abceb02-f2ad-457c-920a-fd8ad3f59cbb', embedding=None, metadata={'정책명': '대전 전세사기피해자 지원', '사업 신청 기간': '상시', '연령': '제한없음', '거주지': '❍ 지원대상 : 전세사기피해를 입은 임차인(특별법의 전세사기피해자등, 특별법 시행전 주택도시보증공사에서 전세피해확인서 발급자) ❍ 지원요건 : 전세사기피해자 결정일 현재 ① 임대차계약을 체결한 피해주택의 소재지가 대전시이고, ② 대전시에 주민등록을 두고 거주하는 자', '운영 기관': '대전 전세피해지원센터', '주관 기관': '대전 전세피해지원센터', '특화 분야': '제한없음', '취업 상태': '제한없음', '학력': '제한없음'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='정책명: 대전 전세사기피해자 지원\n설명: 대전광역시 전세사기피해자 주거안정 및 피해회복 지원\n※ 당해년도 예산이 소진되어 신청하신 지원금은 2025년도에 지급될 예정입니다.\n지원 내용: □ 주거안정지원금(최대 100만원)\nㅇ 가구원수에 따라 차등 지급(1인 가구: 60만원, 2인 가구: 80만원, 3인이상 가구: 100만원)\nㅇ 가구원수는 주민등록에 생계를 같이하는 가족(친족)\n□ 이사비용(공공임대주택 입주시, 최대 100만원)\nㅇ 관내 공공임대주택 입주자가 이사업체에 지불한 비용(이사비, 사다리차 이용비, 에어컨 이전설치비 등)\n□ 월세(민간주택 이주시, 최대 480만원)\nㅇ 경매로 인해 피해주택에서 대전시 새로운주택으로 이사한 피해자가 임대차계약서상 임대인에게 지불한 월차임 (관리비, 공과금 

In [13]:
!pip install InstructorEmbedding
!pip install -U sentence-transformers==2.2.2
!pip install transformers==4.26.0 huggingface-hub==0.23.2



In [20]:
from InstructorEmbedding import INSTRUCTOR
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 모델 초기화
embed_model = INSTRUCTOR("hkunlp/instructor-base")

# 지침 정의
query_instruction = "Represent the user's question for retrieving relevant policies:"
text_instruction = "Represent the policy document for retrieval:"

# 1. 사용자 질문
user_question = "저는 25세 여성이고 대전에서 살고있으며 약 5천만원을 모으고 싶고, 월세 지원 정책을 찾고 있습니다."
query_embedding = embed_model.encode([[query_instruction, user_question]])

# 2. 정책 문서들
policy_documents2 = filtered_docs

# 2-1. 문서 임베딩
document_embeddings = []

for doc in policy_documents2:
    # 메타데이터 문자열 생성
    metadata_str = "\n".join([f"{key}: {value}" for key, value in doc.metadata.items()])
    
    # 텍스트와 메타데이터 병합
    full_text = f"{metadata_str}\n\n{doc.text_resource.text}"
    
    # 임베딩 생성
    embedding = embed_model.encode([["정책 설명:", full_text]])
    document_embeddings.append(embedding)


# 유사도 계산
similarities = [
    cosine_similarity(query_embedding, doc_embed)[0][0] for doc_embed in document_embeddings
]

# 유사도 기반 추천 정렬
sorted_policies = sorted(
    zip(policy_documents2, similarities), key=lambda x: x[1], reverse=True
)

# 결과 출력
print("추천 정책:")
for policy, score in sorted_policies:
    # 'title'이 메타데이터에 포함된 경우 접근
    policy_title = policy.metadata.get("정책명", "제목 없음")
    print(f"- {policy_title} (유사도: {score:.2f})")


load INSTRUCTOR_Transformer
max_seq_length  512
추천 정책:
- 주거안정월세대출 (유사도: 0.94)
- 청년 월세 한시 특별 지원사업 (유사도: 0.94)
- 완주군 청년·신혼부부·다자녀가구 주택 전세자금 대출이자 지원 (유사도: 0.94)
- 광산 청년온(溫)가(家) 운영 (유사도: 0.93)
- 2024년 전세피해 주택임차인 이사비 지원사업 안내 (유사도: 0.93)
- 미혼청년 주거급여 분리지급 (유사도: 0.93)
- 대전 전세사기피해자 지원 (유사도: 0.93)
- 광산구 여성 1인 가구 안심홈세트 지원사업 (유사도: 0.93)
- 신혼부부 청년 임차보증금 지원사업 (유사도: 0.92)
- 1인가구 전월세 안심계약도움서비스 (유사도: 0.92)
- 대학생 주거 지원(서서울관, 동서울관, 청주관) (유사도: 0.92)
- 신혼부부 주택 전세 자금대출이자 지원 (유사도: 0.92)
- 울산 상안지구 행복주택 건립 (유사도: 0.92)
- 성남시 여성1인가구 안심홈세트 지원사업 (유사도: 0.91)


## HuggingFaceEmbedding

The base `HuggingFaceEmbedding` class is a generic wrapper around any HuggingFace model for embeddings. All [embedding models](https://huggingface.co/models?library=sentence-transformers) on Hugging Face should work. You can refer to the [embeddings leaderboard](https://huggingface.co/spaces/mteb/leaderboard) for more recommendations.

This class depends on the sentence-transformers package, which you can install with `pip install sentence-transformers`.

NOTE: if you were previously using a `HuggingFaceEmbeddings` from LangChain, this should give equivalent results.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.3.1
    Uninstalling sentence-transformers-3.3.1:
      Successfully uninstalled sentence-transformers-3.3.1


In [2]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="dunzhang/stella_en_1.5B_v5")  # model_name="BAAI/bge-small-en-v1.5"

modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/169k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

In [8]:
embeddings = embed_model.get_text_embedding("print AHJ")
print(len(embeddings))
print(embeddings[:5])

1024
[-0.027724534273147583, 0.0340331494808197, 0.04841773584485054, 0.021266596391797066, -0.011934561654925346]


## InstructorEmbedding

Instructor Embeddings are a class of embeddings specifically trained to augment their embeddings according to an instruction. By default, queries are given `query_instruction="Represent the question for retrieving supporting documents: "` and text is given `text_instruction="Represent the document for retrieval: "`.

They rely on the `Instructor` and `SentenceTransformers` (version 2.2.2) pip package, which you can install with `pip install InstructorEmbedding` and `pip install -U sentence-transformers==2.2.2`.

In [ ]:
from llama_index.embeddings.instructor import InstructorEmbedding

embed_model = InstructorEmbedding(model_name="hkunlp/instructor-base")

/home/loganm/miniconda3/envs/llama-index/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer


/home/loganm/miniconda3/envs/llama-index/lib/python3.11/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


max_seq_length  512


In [ ]:
embeddings = embed_model.get_text_embedding("Hello World!")
print(len(embeddings))
print(embeddings[:5])

768
[ 0.02155361 -0.06098218  0.01796207  0.05490903  0.01526906]
